#### Notebook Structure

**Part I: Compliance Copilot Report**  
This section is a concise strategic report covering the proposal’s justification, solution overview, key results discussion, identified challenges, conclusion and next steps.

**Part II: Compliance Copilot Demo**  
This section walks through the RAG pipeline steps, prints example outputs, and finally launches the terminal-based Copilot for live testing with a set of curated Q&A examples.

**README.md**  
Contains full technical details on installing dependencies, running the scalable pipeline code, and other execution instructions.


## Part I: Compliance Copilot Report



### 1. Introduction

This section presents the **business rationale**, **strategic justification**, and **solution proposal** for the **Dow Jones AI Compliance Copilot**—a tool designed to **add an intelligence layer** to compliance data that Dow Jones already licenses, processes, and delivers to B2B clients. By embedding these datasets into an RAG-powered Q&A interface, clients can:

- Receive instant, source-cited answers in natural language
- Save dozens of weekly hours on due diligence
- Reduce the risk of missing critical insights
- Make faster, more accurate decisions

Financially, this intelligence layer can **significantly increase the perceived value** of licensed data, unlocking new upsell and recurring revenue opportunities.

### 2. Strategic Justification & Market Opportunity

#### 2.1 Why B2B Compliance First?
- The **Governance, Risk & Compliance (GRC)** market is valued at **$62.9 billion** in 2024 with a **13.2% CAGR** through 2030 [1].
- The **RegTech** subsector grows from **$4.7 billion** in 2024 to **$29 billion** by 2034 at a **20% CAGR** [2].
- B2B compliance solutions command **ARPU of $50K–$500K/year**, far above B2C offerings [3].

Focusing initially on corporate compliance clients maximizes ROI, tapping a fast-growing market with buyers willing to invest in high-value solutions.

### 2.2 Existing Dow Jones Offerings
- **RiskFeeds**: Structured feeds (sanctions, PEPs, adverse media) in **XML, CSV, JSON** formats for screening and reporting [4].
- **Integrity Check (with Xapien)**: Entity due diligence reports based on entity name and country, cutting analysis from **days to minutes** [5].

**How the Copilot Differs**:
- **Complementary** to RiskFeeds and Integrity Check, adding open-ended Q&A across all licensed data.
- Returns **source-cited answers** instead of fixed reports or raw feeds.
- Provides **ad hoc insights** to support immediate decision-making, beyond standard outputs.

### 3. Solution Proposal

#### 4.1 Data Acquisition
- **Production**: Clients continue to license **proprietary Dow Jones data** (sanctions, PEPs, filings, adverse media) via XML/CSV/JSON and APIs.
- **POC**: For this proof of concept, **publicly sourced documents** were extracted and processed from the internet to simulate the same data structure.

#### 4.2 Pipeline
```plaintext
1. Licensed Dow Jones datasets
2. Ingestion & Parsing (PDF, CSV/XML, HTML)
3. Vector Indexing (embeddings)
4. Agent AI + LLM (semantic search + generation)
5. Source-cited answers in Q&A interface
```

#### 4.3 Key POC Components
- **RAG**: Combines vector search with LLM for grounded responses.
- **Prompt Engineering**: Domain-specific prompt tuning.
- **Model Flexibility**: Support for GPT-4, Claude, and on-premise Ollama models.

#### 4.4 README & Usage Instructions
The **README** in the repository guides:
- **Installation & Setup**: Virtual environment, dependencies, local models.
- **Pipeline Execution**: Document download (`docs/fetch_data.py`), semantic index build (`src/build_index.py`), CLI Q&A (`main.py`, `backend.py`).
- **Notebook Demo**: `compliance_demo_report.ipynb` walks through interactive tests with example questions and source-cited answers.

#### 4.5 Test Questionnaire
Because local indexing was resource-intensive and only a small document set was indexed, the file **`compliance_suggestion_questions.md`** includes curated test questions referencing the indexed context. This accelerated testing by covering scenarios such as:
- Entity mention lookups in sanctions and reports
- Queries on recent regulatory recommendations
- Verifications of company-risk alerts 

### 4. Next Steps
1. **Infrastructure**: Migrate local storage to AWS S3/Azure Blob/GCP Buckets.
2. **OCR Integration**: Add Tesseract or Amazon Textract for scanned PDFs.
3. **Structured Validation**: Define precision/recall metrics and gather expert feedback.
4. **Deployments**: REST API (FastAPI), front-end (React/Streamlit), cloud deployment (Vercel, Hugging Face).
5. **Multi-tenant & Customization**: Client-specific histories and permissions.

### 6. Conclusion
The **Dow Jones AI Compliance Copilot** introduces a conversational intelligence layer atop already licensed data, delivering rapid, cited, and contextual insights. This evolution drives **new recurring revenue**, **higher margins**, and **enhances Dow Jones' leadership** in the RegTech space.

### References
1. Grand View Research, “eGRC Market Size, Share & Trends Analysis Report,” 2024.
2. Mordor Intelligence, “RegTech Market Forecast to 2034.”
3. Gartner, “Enterprise GRC Buyer Insights,” 2023.
4. The Wealth Mosaic, “Dow Jones RiskFeeds Overview.”
5. Xapien & Dow Jones Partnership Announcement, 2022.

----------------------------------------------------------------------

## Part 2: Compliance Copilot Demo 

### 1. Best Practices for Demo Usage

- **Ask objective, specific questions** related to compliance.  
- **Avoid vague or overly broad questions.**  
- **Verify any uncertain or ambiguous answers**, as the system is restricted to indexed content and is parameterized to minimize hallucinations—watch for any extrapolation.  
- **Consult `compliance_suggestion_questions.md`** for a sample of supported questions.  

_In this demo notebook, you will first see a concise overview of the pipeline steps and outputs. After that, you’ll test the Copilot interactively via the terminal interface._ 

### 2. Load and Preview Raw Documents

The `docs/fetch_data.py` simulates how compliance documents would be ingested for the POC by:

1. **Downloading** various public filings, advisories, and sanction lists (SEC, OFAC, FATF, FinCEN, EU) into `data/raw`.  
2. **Handling failures** (403/404) by adding files manually when needed.  
3. **Note**: In a real deployment, documents (PDF, CSV, TXT, etc.) would be loaded directly from authenticated internal storage or cloud buckets, not scraped from public websites.

In [1]:
import os

RAW_PATH = "data/raw"
print(" === Raw files: ===")
for fname in os.listdir(RAW_PATH):
    print("-", fname) 

 === Raw files: ===
- apple_10k_2022.txt
- eu_annex2_sanctions.pdf
- fatf_annual_report_2022-2023.pdf
- fatf_assessment_methodology_2022.pdf
- fatf_effectiveness_compliance_report_2022.pdf
- fatf_procedures_mutual_evaluations_2022.pdf
- fatf_universal_procedures.pdf
- fincen_advisory_corruption_2022.pdf
- fincen_advisory_elder_exploitation_2022.pdf
- fincen_alert_pig_butchering_2023.pdf
- fincen_alert_russian_elites_2022.pdf
- fincen_ransomware_advisory.pdf
- microsoft_10k_2022.txt
- ofac_sdn_list.csv


### 3. Embedding and FAISS Index

The `build_vector_store` function (in lieu of rerunning the full `build_index.py` pipeline) allows you to update the FAISS index with any new or changed documents:

1. **Load documents** from `data/raw` using `load_documents`.  
2. **Embed and store** them in `data/index` via `embed_and_store` from `embedder.py`.  



The `embed_and_store` function handles document chunking, embedding, and FAISS index creation:

1. **Chunking**: Splits each document into 512-char chunks (no overlap) using `RecursiveCharacterTextSplitter`.  
2. **Embedding Model**: Uses `sentence-transformers/paraphrase-albert-small-v2` (normalized embeddings via `HuggingFaceEmbeddings`) for efficient, local inference.  
3. **Index Creation**: Builds a FAISS index from the chunked texts and associated metadata.  
4. **Persistence**: Saves the FAISS index to `data/index` for fast retrieval.

> ** Attention:** This process recreates the index in `data/index`.  
> Do **not** rerun unless you have added or modified documents in `data/raw`; otherwise you may overwrite your existing index.

In [ ]:
#from src.build_index import main as build_index

# This module creates the index in the data/index directory. 
# Its not recommended to run now it again unless you change the documents in the data/raw indexing another ones in data/index directory.
#build_index() 

### 4. Inspect FAISS Index Contents

The `inspect_index` function lets you peek “under the hood” of your FAISS vector store by:
1. Loading the same embedding model used for indexing (`paraphrase-albert-small-v2` with normalized embeddings).
2. Deserializing the FAISS index from `data/index`.
3. Iterating over each document in the internal store to build a summary:
   - **doc_id**: 1-based counter  
   - **source**: original filename or URL metadata (or `"unknown"`)  
   - **preview**: first ~300 characters of the document text


In [ ]:
from inspect_index import inspect_index

inspect_index("data/index") 

c:\Users\Ana\Desktop\compliance_RAG\inspect_index.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
c:\Users\Ana\Desktop\compliance_RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'doc_id': 1,
  'source': 'data\\raw\\eu_annex2_sanctions.pdf',
  'preview': '1 \nAnnex II – Sanctions-related commitments   \n \nThe sequence of implementation of the commitments d etailed in this Annex is \nspecified in Annex V (Implementation Plan) to this Joint Comprehensive Plan of \nAction (JCPOA). \n  \n \nA.  European Union 1 \n \n1.  The EU and EU Member States commit to termi'},
 {'doc_id': 2,
  'source': 'data\\raw\\eu_annex2_sanctions.pdf',
  'preview': 'specified in Sections 1.1-1.10 below, to terminate all provisions of \nCouncil Decision 2010/413/CFSP (as subsequently ame nded), as \nspecified in Sections 1.1-1.10 below, and to termin ate or amend \nnational implementing legislation as required, in a ccordance with \nAnnex V:  \n \n1.1. \n Financial, ba'},
 {'doc_id': 3,
  'source': 'data\\raw\\eu_annex2_sanctions.pdf',
  'preview': '30a, 30b and 31 of Council Regulation (EU) No 267/2 012); \n \n1.1.2. \n Sanctions on banking activities (Article 11 of Coun cil Decision 

### 5. Run a Sample QA Interaction

The `backend.py` module defines how to construct and run the RAG-powered Q&A chain:

1. **QA_PROMPT**  
   - Custom prompt template that instructs the model to answer using only the provided context and to return a polite fallback if no answer is found.

2. **build_qa_chain(index_path: str)**  
   - **Loads** the FAISS index from `index_path` with the same embedding model (`paraphrase-albert-small-v2`).  
   - **Initializes** the local Ollama LLM (`tinyllama`) with controlled temperature, top-p, and repeat penalty.  
   - **Creates** a `RetrievalQA` chain that retrieves relevant passages and generates answers via the prompt.

3. **run_qa_app(index_path: str)**  Starts a **terminal-based loop**: prompts the user for questions, invokes the QA chain, prints answers, and exits on “exit”/“quit”. 

## Using the Copilot: Terminal & Notebook 

**1. Terminal Q&A Session**  
Launch the interactive CLI in your terminal (not in a notebook cell):

**Ask:** Type your question at the `>` prompt and press **Enter**.

**Answer:** You’ll see a source-cited response in the terminal.

**Exit:** Press **Esc** or type `exit`/`quit`.

**Tip:** Use the curated questions in `compliance_suggestion_questions.md` for best coverage.


In [ ]:
from backend import run_qa_app

#THIS IS RUNNING AT THE TERMINAL, NOT IN THE CELLS
run_qa_app("data/index")  

Loading FAISS index from: data/index
=== Starting local LLM via Ollama ===
=== System is ready. Ask your compliance questions below ===


=== Answer:
Sorry, I couldn't find any information based on the provided context about whether you can engage in activities with or provide financial services and other goods and services used in connection with Bandar Abbas port operator (including the port operator of Bandar Abbas) or sale, leasing, and provision of vessels to international financial institutions.


=== Answer:
Answer: Cryptocurrency mixers are used in ransomware schemes to convert virtual currencies (VCC) into a form that can be traded or exchanged for real money. In these schemes, criminals create a new wallet or address for each transaction and use the mixer service to combine transactions from multiple sources and conceal their identity. This allows them to launder money and evade detection by law enforcement agencies.


=== Answer:
The provided context states that in October 2

## Tested Questions and Observations

Below are the initial test questions executed against the POC, along with high-level notes on model behavior:

1. **OLÁ**  
   - **Purpose**: Show when the model has no answer.  
   - **Observation**: The model responded with a fallback message indicating no relevant content found.

2. **How are cryptocurrency mixers used in ransomware schemes?**  
   - **Purpose**: Evaluate domain knowledge.  
   - **Observation**: Model retrieved and cited passages explaining how mixers obfuscate illicit funds in ransomware workflows.

3. **What are the risks associated with cryptocurrency mixers?**  
   - **Purpose**: Assess contextual risk analysis.  
   - **Observation**: Model generated a list of risks (e.g., lack of traceability, regulatory non-compliance) with proper citations.

4. **How are cryptocurrency mixers used in ransomware schemes?** (repeated)  
   - **Purpose**: Test consistency and improvement with follow-up.  
   - **Observation**: The answer expanded with additional context, demonstrating iterative refinement.

5. **What best practices does FinCEN recommend for documenting EFE in SAR reports?**  
   - **Purpose**: Validate coverage of regulatory guidance.  
   - **Observation**: Model cited FinCEN SAR manual guidance, detailing recommended documentation practices for Exploitation of a Financial Enterprise (EFE).
